In [23]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
#맷플랏립 그래프 주피터에서 바로 출력하기
import seaborn as sns

from scipy import stats
from scipy.stats import t, norm, chi2, chi2_contingency
import re
import time

from matplotlib import rc
rc('font', family='Malgun Gothic')      #한글 폰트설정
plt.rcParams['axes.unicode_minus']=False      #마이너스 부호 출력 설정

In [4]:
from sklearn.datasets import load_iris
from sklearn import datasets

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score

In [22]:
from sklearn import tree
from sklearn.datasets import make_classification

from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [1]:
### 일괄 전처리 ###

def name(df):
    arr = df.Name.values
    df['Name'] = [re.search(',\s([A-Za-z\s]+)[.]\s*', i)[1] for i in arr]   #직업/특징 데이터 뽑기
    return df

def nullskip(df):   #널값 처리
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    maxvalue = df['Embarked'].value_counts(dropna=True).idxmax()
    df['Embarked'].fillna(maxvalue, inplace=True)
    return df

def get_age(age):
    cat = ''
    if age < 5: cat='Baby'
    elif age < 15: cat= 'Child'
    elif age < 21: cat= 'Young Adult'
    elif age < 49: cat='Adult'
    elif age < 70: cat='Old'
    else: cat = 'Elder'
    return cat

def get_cate(fare):
    cat=''
    if fare<=15: cat='저가'
    elif fare<=60: cat='중가'
    elif fare<=200: cat='고가'
    else: cat='프리미엄'
    return cat

def drop_col(df):   #불필요한 속성(칼럼) 제거
    df['Family'] = df.SibSp + df.Parch
    df['Cabin'] = df['Cabin'].str[:1]
    
    df.drop(['PassengerId','Ticket','SibSp','Parch'], axis=1, inplace=True)
    return df

def format_enc(df):   #레이블 인코딩 수행
    features = ['Cabin','Sex','Embarked', 'Family', 'Age','Fare','Name']
    for i in features:
        le = LabelEncoder()
        le = le.fit(df[i])
        df[i] = le.transform(df[i])      
    return df
####################################################################
def trans(df):   #위에서 만들어둔 함수들 호출
    df = name(df)
    df = nullskip(df)
    df['Age'] = df.Age.apply(lambda x: get_age(x))
    df['Fare'] = df.Fare.apply(lambda x: get_cate(x))
    df = drop_col(df)
    df = format_enc(df)
    return df

###  앙상블 학습

* 앙상블 학습의 유형은 보팅, 배깅, 부스팅 세가지로 나눌 수 있으며 이외에도 스태깅을 포함한 다양한 앙상블 방법이 있다.
* 보팅의 경우 서로 다른 알고리즘을 가진 분류기를 결합하는 것이고 배깅의 경우 각각의 분류기각 모두 같은 유형의 알고리즘 기반이다.
* 정형 데이터의 예측 분석 영역에서는 매우 높은 예측 성능. Bagging 과 Boosting
* 배깅 방식의 대표인 Random Forest는 뛰어난 예측 성능, 상대적으로 빠른 수행시간, 유연성 등으로 애용.
* 부스팅의 효시는 Gradient Boosting, 한 단계 발전시키면서도 시간 단축시킨 XgBoost, LightGBM이 정형 데이터의 분류 영역에서 
  활용도 확대
* 앙상블의 앙상블이라고 불리는 스태킹 기법
* 앙상블의 기본 알고리즘은 결정 트리

### Voting Classifier
- 하드 보팅 : 다수결 원칙, 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정
- 소프트 보팅 : 분류기들의 레이블 값 결정 확률을 모두 더해서 평균하고 이들 중 가장 높은 레이블 값을 최종 보팅 결과값으로 선정

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import VotingClassifier
cancer = load_breast_cancer()

# 개별 모델은 로지스틱 회귀와 KNN임
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델은 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], 
                          voting='soft')

x_train, x_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                test_size=0.2, random_state=156)

# VotingClassifier 학습, 예측, 평가
vo_clf.fit(x_train, y_train)
pred = vo_clf.predict(x_test)
print(f'보팅 분류기 정확도: {accuracy_score(y_test, pred):.4f}')

# 개별 모델의 학습, 예측, 평가
classifiers = [lr_clf, knn_clf]
for i in classifiers:
    i.fit(x_train, y_train)
    pred = i.predict(x_test)
    class_name = i.__class__.__name__
    print(f'{class_name} 정확도: {accuracy_score(y_test, pred):.4f}')

보팅 분류기 정확도: 0.9474
LogisticRegression 정확도: 0.9386
KNeighborsClassifier 정확도: 0.9386


In [20]:
cancer

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

### 랜덤 포레스트(Random Forest)
- 앙상블 기법 중 하나로, 의사결정나무(Decision Tree)를 기반으로 한 분류 및 회귀 알고리즘입니다. 랜덤 포레스트는 <<<여러 개의 의사결정나무를 조합>>>하여 강력한 예측 모델을 구축하고, 과적합을 방지하며 일반화 성능을 향상시킵니다.
- 랜덤 포레스트는 다양한 분류 및 회귀 문제에 적용할 수 있는 강력한 알고리즘으로 알려져 있으며, 데이터셋의 특성과 목표에 맞게 사용될 수 있습니다.

#### 랜덤 포레스트의 동작 원리

- 데이터 샘플링 (부트스트래핑(bootstrapping))<br>
: 원본 데이터에서 무작위로 <<<중복을 허용>>>하여 샘플을 선택합니다. 이렇게 선택된 샘플들을 사용하여 각각의 의사결정나무를 학습시킵니다. 이러한 방식을 부트스트래핑(bootstrapping)이라고 합니다.

- 특징 선택<br>
: 각 의사결정나무의 학습 과정에서 특징 선택 시 무작위로 <<<일부 특징만을 고려>>>합니다. 이는 "의사결정나무 간의 다양성"을 증가시켜 모델의 예측력을 향상시키는 역할을 합니다.

- 의사결정나무 학습<br>
: 선택된 샘플과 특징을 사용하여 의사결정나무를 학습시킵니다. 각 의사결정나무는 주어진 데이터에 대해 분할 기준과 분기를 결정하여 예측 규칙을 생성합니다.

- 예측 결합<br>
: 학습된 모든 의사결정나무를 사용하여 새로운 데이터의 예측값을 도출합니다. 분류 문제에서는 다수결 투표를 통해 가장 많은 표를 받은 클래스가 최종 예측값이 됩니다. 회귀 문제에서는 평균값을 사용합니다.

#### 랜덤 포레스트 장점

- 과적합 방지: <br>
랜덤 포레스트는 의사결정나무의 과적합 문제를 완화시킵니다. 샘플링과 특징 선택의 무작위성을 통해 다양한 의사결정나무를 조합하고, 이들의 예측을 평균화함으로써 일반화 성능을 향상시킵니다.

- 변수 중요도 제공: <br>
랜덤 포레스트는 변수의 중요도를 계산할 수 있습니다. 각 의사결정나무에서 변수의 사용 빈도나 분산 기준에 따라 중요도를 측정하고, 이를 모든 의사결정나무에서 평균화하여 변수의 상대적 중요도를 얻을 수 있습니다. 이를 통해 데이터에서 어떤 변수가 예측에 가장 큰 영향을 미치는지를 알 수 있습니다.

- 안정성과 신뢰성: <br>
랜덤 포레스트는 여러 개의 의사결정나무를 결합한 모델이므로, 개별 의사결정나무의 오류나 노이즈에 덜 민감합니다. 이를 통해 안정적이고 신뢰성 있는 예측을 제공할 수 있습니다.

- 다양한 데이터 타입 처리: <br>
랜덤 포레스트는 범주형 데이터와 연속형 데이터 모두를 처리할 수 있습니다. 범주형 변수의 경우 원-핫 인코딩 등의 전처리 과정이 필요하지 않습니다.

- 비교적 빠른 학습과 예측 속도: <br>
의사결정나무의 학습과 예측이 <<<병렬로 수행>>>되므로, 데이터가 크거나 차원이 높아도 상대적으로 "빠른 학습 및 예측 속도"를 보장할 수 있습니다.

#### 랜덤 포레스트 단점
- 랜덤 포레스트는 의사결정나무의 단점인 해석력이 다소 떨어진다는 점을 가지고 있습니다. 
<br>여러 개의 의사결정나무를 조합하므로 최종 모델의 해석이 어려울 수 있습니다. 
- 또한, 랜덤 포레스트는 모델 구성을 위해 많은 수의 의사결정나무를 사용해야 하므로, 모델의 복잡성과 메모리 사용량이 증가할 수 있습니다.

#### 과제(1): 타이타닉 타이타닉 데이터셋에 대해 필요한 전처리 수행 후 랜덤포레스트 알고리즘을 적용, 평가, 성능개선(하이퍼 파라미터 튜닝/ 교재 218p)을 수행
1. 에스티메이터: 너무 많이하면 메모리 부족 (숫자를 점점 키우면서 결과 보기/ 어느 순간 정확도가 떨어지면 그 즈음에서 멈추기)
2. 나머진 디시전 트리랑 똑같음

#### 과제(2): 과제1로부터 변수 중요도(피처임포턴스-5개만)를 도출하고 시각화하세요

(머신러닝 - 타이타닉에 과제한 거 있음)

## GBM(Gradient Boosting Machine)
- 부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식
- 가중치 업데이트를 경사 하강법(Gradient Descent)를 이용한다.
- 분류는 물론이고 회귀도 가능

#### 파라미터 : n_estimators, max_depth, max_features...
- loss : 경사하강법에서 사용할 비용함수 지정. (기본값=deviance)
- learning_rate : GBM이 학습할 때마다 적용할 학습률.오류값 보정 시 적용하는 계수로 0 ~ 1 사이의 값 지정. 기본값은 0.1. <br> 작게(촘촘하게) 설정하면 예측성능이 높아지나 수행시간이 오래 걸리고, 큰 값을 적용하면 예측 성능이 떨어질 가능성이 높으나 빠른 수행이 가능. n_estimator와 상호 보완적으로 조합해 사용
- n_estimator : weak learner(약한 학습기)의 개수
- subsample : weak learner가 학습에 사용하는 데이터의 샘플링 비율. (기본값=1)

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
import time

df = pd.read_csv(r'C:\Users\h\hmkd1\mc_data\train.csv')
df = trans(df)   # 일괄 전처리: trans() 함수

ydf = df.Survived
xdf = df.drop('Survived', axis=1)

x_train, x_test, y_train, y_test = train_test_split(xdf, ydf, test_size=0.2, random_state=11)
start = time.time()

gb = GradientBoostingClassifier(random_state=0)
gb.fit(x_train, y_train)

pred = gb.predict(x_test)
acc = accuracy_score(y_test, pred)

print(f'GBM 정확도: {acc:.4f}')
print(f'GBM 수행시간: {(time.time()-start):.2f}초')

GBM 정확도: 0.8715
GBM 수행시간: 0.07초


In [24]:
from sklearn.model_selection import GridSearchCV
params={
    'n_estimators':[50,100,200,300],
    'learning_rate':[0.05, 0.1, 0.15, 0.2]
}
grid_cv = GridSearchCV(gb, param_grid=params, cv=5, verbose=1)
grid_cv.fit(x_train, y_train)

print('최적 하이퍼 파라미터:', grid_cv.best_params_)
# Fitting 5 folds for each of 16 candidates, totalling 80 fits
# 5개 폴드를 각 파라미터별(16번) 진행 = 총 80회
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
최적 하이퍼 파라미터: {'learning_rate': 0.15, 'n_estimators': 50}
최고 예측 정확도: 0.8202


In [21]:
# GridSearchCV를 이용하여 최적으로 학습된 estimator로 predict 수행. 
gb_pred = grid_cv.best_estimator_.predict(x_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(f'GBM 정확도: {gb_accuracy:.4f}')

GBM 정확도: 0.8659


(3) 앙상블 분류기
-여러 분류기를 생성하고 예측을 결합함으로서 보다 정확한 최종 예측 도출
-쉽고 편하고 강력한 성능이 특징

-보팅과 배깅은 여러 분류기가 투표를 통해 최종 예측 결과 결정
   ㄴ보팅: 서로 다른 알고리즘을 가진 분류기 결합
   ㄴ배깅: 같은 유형의 알고리즘, 다른 데이터 샘플링 (ex. 랜덤 포레스트)
-부스팅은 여러 분류기가 순차적으로 학습을 수행하되, 앞선 틀린 예측 데이터를 다음 분류기에게 가중치를 부여하며 학습과 예측을 진행함

<보팅>
-하드보팅: 다수결의 원칙과 비슷
-소프트 보팅: 분류기들의 레이블 값(정답) 결정 확률을 모두 더하고 평균 내, 이중 확률이 가장 높은 레이블 값을 최종 보팅 (성능 더 좋음)

<배깅>
-같은 모델에서 여러개의 분류기를 만들어 보팅으로 최종 결정
-같은 알고리즘 모델, 다른 데이터 집합
-부트스트래핑 분할 방식: 할당되는 데이터는 원본 데이터를 샘플링 해 추출함

<부스팅>
-가중치를 부스팅하며 학습을 진행하기에 부스팅 방식
-예측 성능 매우 뛰어남

###  앙상블 학습

* 앙상블 학습의 유형은 보팅, 배깅, 부스팅 세가지로 나눌 수 있으며 이외에도 스태깅을 포함한 다양한 앙상블 방법이 있다.
* 보팅의 경우 서로 다른 알고리즘을 가진 분류기를 결합하는 것이고 배깅의 경우 각각의 분류기각 모두 같은 유형의 알고리즘 기반이다.
* 정형 데이터의 예측 분석 영역에서는 매우 높은 예측 성능. Bagging 과 Boosting
* 배깅 방식의 대표인 Random Forest는 뛰어난 예측 성능, 상대적으로 빠른 수행시간, 유연성 등으로 애용.
* 부스팅의 효시는 Gradient Boosting, 한 단계 발전시키면서도 시간 단축시킨 XgBoost, LightGBM이 정형 데이터의 분류 영역에서 
  활용도 확대
* 앙상블의 앙상블이라고 불리는 스태킹 기법
* 앙상블의 기본 알고리즘은 결정 트리

### Voting Classifier
- 하드 보팅 : 다수결 원칙, 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정
- 소프트 보팅 : 분류기들의 레이블 값 결정 확률을 모두 더해서 평균하고 이들 중 가장 높은 레이블 값을 최종 보팅 결과값으로 선정

### Voting Classifier

In [ ]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

In [ ]:
# 개별 모델은 로지스틱 회귀와 KNN 임. 
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기 
vo_clf = VotingClassifier( estimators=[('LR',lr_clf),('KNN',knn_clf)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2 , random_state= 156)

# VotingClassifier 학습/예측/평가. 
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

# 개별 모델의 학습/예측/평가.
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train , y_train)
    pred = classifier.predict(X_test)
    class_name= classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test , pred)))

## 4.4 Random Forest

In [ ]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [ ]:
import pandas as pd

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 결정 트리에서 사용한 get_human_dataset( )을 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

# 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy))

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}
# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
rf_clf1 = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=8, \
                                 min_samples_split=8, random_state=0)
rf_clf1.fit(X_train , y_train)
pred = rf_clf1.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns  )
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)
fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig('rf_feature_importances_top20.tif', format='tif', dpi=300, bbox_inches='tight')

## 4.5 GBM(Gradient Boosting Machine)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))


In [ ]:
### 아래는 책에서 설명드리지는 않지만 GridSearchCV로 GBM의 하이퍼 파라미터 튜닝을 수행하는 예제 입니다. 
### 사이킷런이 1.X로 업그레이드 되며서 GBM의 학습 속도가 현저하게 저하되는 문제가 오히려 발생합니다. 
### 아래는 수행 시간이 오래 걸리므로 참고용으로만 사용하시면 좋을 것 같습니다. 

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100, 500],
    'learning_rate' : [ 0.05, 0.1]
}
grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2 ,verbose=1)
grid_cv.fit(X_train , y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
# GridSearchCV를 이용하여 최적으로 학습된 estimator로 predict 수행. 
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))